In [1]:
import tensorflow as tf

/home/userhadoop/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
def init_weight(shape):
    init_weight_rand = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_weight_rand)

In [5]:
def init_bias(shape):
    init_bias_rand = tf.constant(0.1,dtype=tf.float32,shape=shape)
    return tf.Variable(init_bias_rand)

In [6]:
def conv_2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

In [7]:
def pool_max(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [8]:
def convolution_layer(input_x,shape):
    w=init_weight(shape)
    b=init_bias([shape[3]])
    return tf.nn.relu(conv_2d(input_x,w)+b)

In [9]:
def normal_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    w = init_weight([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, w) + b

In [10]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [11]:
x_image = tf.reshape(x,[-1,28,28,1])

In [12]:
convo_1 = convolution_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = pool_max(convo_1)

In [13]:
convo_2 = convolution_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = pool_max(convo_2)

In [14]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_layer(convo_2_flat,1024))

In [15]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [16]:
y_pred = normal_layer(full_one_dropout,10)

In [17]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [20]:
init = tf.global_variables_initializer()

In [22]:
steps = 500

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
           
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1269


Currently on step 100
Accuracy is:
0.8307


Currently on step 200
Accuracy is:
0.8983


Currently on step 300
Accuracy is:
0.9254


Currently on step 400
Accuracy is:
0.9389


